读取数据集、定义神经网络
===========================

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

超参数
===============
训练神经网络时的三个基本超参数：
-   **Number of Epochs** - 遍历数据集的次数
-   **Batch Size** - 每次计算梯度时使用的样本数量
-   **Learning Rate** - 学习率.


In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

Optimization Loop
=================

每个优化神经网络的循环被称为 **epoch**，每个 epoch 由两部分组成:

-   **训练** - 遍历训练集，优化网络参数。
-   **验证** - 遍历测试集，检查网络性能是否提升。

Pytorch实现了常用的[损失函数](https://pytorch.org/docs/stable/nn.html#loss-functions)：

[nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)
(Mean Square Error)用于回归问题。

[nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)
(Negative Log Likelihood)用于分类问题。

[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)
是交叉熵损失，结合了 `nn.LogSoftmax` 与 `nn.NLLLoss`。

In [ ]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

优化器
=========
所有优化计算都被封装在`optimizer` 对象中，
这里使用了随机梯度下降(SGD optimizer)。

Pytorch还内置了多种 [其它优化器](https://pytorch.org/docs/stable/optim.html)
，例如Adam和Rmsprop等。

初始化optimizer对象时，需要传入待优化的模型参数和学习率。

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练循环内，优化过程分为三步:

-调用`optimizer.zero_grad()`重置模型参数的梯度。默认情况下梯度会叠加，为了防止重复计算，需要在每次迭代中显式地将它们清零。

-调用`loss.backward()`来反向传播预测损失。PyTorch记录了损失关于每个参数的梯度。

-调用`optimizer.step()`，使用反向传播中收集的梯度来调整参数。

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, scheduler):
    size = len(dataloader.dataset)
    # 将model设为训练模式，对batch normalization和dropout很重要
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 计算模型输出和损失
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # 将model设为测试模式，对batch normalization和dropout很重要
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # 测试模型时使用 torch.no_grad(), 使测试时不计算梯度
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2,
                                                                     eta_min=learning_rate * 1e-3)
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    scheduler.step()
print("Done!")